In [24]:
images_folder = '../../datasets/images_raw/'
txt_annotations_folder = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/annotations/txt_annotations'
images_subfolders = ['backgrounds', 'jeremy', 'mwitt', 'openimagesv7', 'rf1', 'rf2']
val_folder = '../../datasets/sharktrack_mvds/v4/val'
train_folder = '../../datasets/sharktrack_mvds/v4/train'
# test_folder = '../mvd/test'
annotations_path = '../../datasets/annotations/all_annotations.csv'

# Convert Annotations to YOLO Format

In [2]:
import pandas as pd
import os

In [3]:
annotations = pd.read_csv(annotations_path)
# Filter the annotations that have in the Augmentation string the word cutout
# note some of the cells have NaN values, so we need to convert them to string
# annotations = annotations[annotations['Augmentation'].astype(str).str.contains('Cutout')]
annotations.head()

,Filename,Family,Genus,Species,ymin,xmin,xmax,ymax,Augmentation,Source
0,mwitt_2021_01_LGX030003.MP4_0005725.jpg,Carcharhinidae,Carcharhinus,perezi,0.500926,0.403125,0.506250,0.560185,NaN,mwitt
1,mwitt_2021_01_LGX050002.MP4_0016230.jpg,Carcharhinidae,Carcharhinus,acronotus,0.171296,0.018750,0.251042,0.275000,NaN,mwitt
2,mwitt_2021_01_LGX030004.MP4_0015520.jpg,Carcharhinidae,Carcharhinus,acronotus,0.105824,0.637896,0.962302,0.375667,NaN,mwitt
3,mwitt_2021_01_LGX050004.MP4_0014447.jpg,Carcharhinidae,Carcharhinus,perezi,0.192593,0.408333,0.613542,0.360185,NaN,mwitt
4,mwitt_2021_01_LGX020009.MP4_0019403.jpg,Carcharhinidae,Carcharhinus,perezi,0.551852,0.772917,0.903646,0.695370,NaN,mwitt


In [14]:
# List all jpeg filenames in the images_folder/images_subfolders

# Get all the images in the images folder
all_images = []
for folder in images_subfolders:
    for root, dirs, files in os.walk(os.path.join(images_folder, folder)):
        for file in files:
            if file.endswith(".jpg"):
                all_images.append(file)

In [21]:
# For each annotation, create a new filename ../yolov8/{Filename - '.jpg'}.txt of the following format:
# <class> <x_centre> <y_centre> <width> <height>
# where class is 0 and the rest are calculated from ymin, xmin, ymax, xmax

# Please note, the annotations might have multiple rows for the same Filename,
# in case there are multiple annotations. In this case, you should create one file
# with multiple lines
# 0 <x_centre_1> <y_centre_1> <width_1> <height_1>
# 0 <x_centre_2> <y_centre_2> <width_2> <height_2>
for index, row in annotations.iterrows():
    filename = row['Filename']
    xmin = row['xmin']
    xmax = row['xmax']
    ymin = row['ymin']
    ymax = row['ymax']

    # Coordinates, rounded to 4 decimal places
    width = xmax - xmin
    height = ymax - ymin
    x_centre = xmin + width / 2
    y_centre = ymin + height / 2
    width = round(width, 4)
    height = round(height, 4)
    x_centre = round(x_centre, 4)
    y_centre = round(y_centre, 4)

    txt_file = f'{txt_annotations_folder}/{filename[:-4]}.txt'

    if filename in all_images:
      # If the file already exists, append the new line
      if os.path.exists(txt_file):
          print('path already exists')
          with open(txt_file, 'a') as f:
              f.write(f'0 {x_centre} {y_centre} {width} {height}\n')
      else:
          with open(txt_file, 'w') as f:
              f.write(f'0 {x_centre} {y_centre} {width} {height}\n')

path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists
path already exists


# Move Images and Txt Data

In [31]:
import os
import random
import shutil

sample = 500

for folder in ['rf3']:
    path = os.path.join(images_folder, folder)
    print(path)
    images = [ f for f in os.listdir(path) if f.endswith('.jpg') ]
    print(len(images))
    random.shuffle(images)

    if sample:
        images = images[:sample]

    # split the images
    val = images[:int(len(images)*0.2)]
    train = images[int(len(images)*0.2):]
    # train = images[int(len(images)*0.2):int(len(images)*0.8)]
    # test = images[int(len(images)*0.8):]

    print(f'val {len(val)}')
    print(f'train {len(train)}')
    # print(len(test))

    # Copy images to val folder
    for image in val:
        shutil.copy(os.path.join(path, image), os.path.join(val_folder, 'images', image))
        txt_file = f'{image[:-4]}.txt'
        if os.path.exists(os.path.join(txt_annotations_folder, txt_file)):
            shutil.copy(os.path.join(txt_annotations_folder, txt_file), os.path.join(val_folder, 'labels', txt_file))

    # Copy images to train folder
    for image in train:
        shutil.copy(os.path.join(path, image), os.path.join(train_folder, 'images', image))
        txt_file = f'{image[:-4]}.txt'
        if os.path.exists(os.path.join(txt_annotations_folder, txt_file)):
            shutil.copy(os.path.join(txt_annotations_folder, txt_file), os.path.join(train_folder, 'labels', txt_file))

# Copy images to test folder
# for image in test:
#     shutil.copy(os.path.join(images_folder, image), os.path.join(test_folder, 'images', image))
#     txt_file = f'{image[:-4]}.txt'
#     if os.path.exists(os.path.join(images_folder, txt_file)):
#         shutil.copy(os.path.join(images_folder, txt_file), os.path.join(test_folder, 'labels', txt_file))

# check the number of images in each folder
print(f"val imges: {len(os.listdir(val_folder + '/images'))}")
print(f"train imges: {len(os.listdir(train_folder + '/images'))}")
print(f"val labels: {len(os.listdir(val_folder + '/labels'))}")
print(f"train labels: {len(os.listdir(train_folder + '/labels'))}")
# print(len(os.listdir(test_folder)))

../../datasets/images_raw/rf3
3527
val 100
train 400
val imges: 1446
train imges: 5803
val labels: 1339
train labels: 5362


imges: 1347
imges: 5400
imges: 1234
imges: 4967

In [4]:
background_images = './openimagesv7_background_train.txt'

In [7]:
images_folder = ['../mvd/test/images/', '../mvd/train/images/', '../mvd/val/images/']

# Copy all images whose filename is in openimagesv7_background_train.txt to ../images_raw/backgrounds
import shutil
import os


for folder in images_folder:
    images = os.listdir(folder)
    for image in images:
        if image.startswith('bkg_'):
            shutil.copyfile(os.path.join(folder, image), os.path.join('../images_raw/backgrounds', image))

In [ ]:
# List all jpeg filenames in the images_folder/images_subfolders

# Get all the images in the images folder
all_images = []
for folder in images_subfolders:
    for root, dirs, files in os.walk(os.path.join(images_folder, folder)):
        for file in files:
            if file.endswith(".jpg"):
                all_images.append(file)